<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/reinforcementlearning_value_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gymnasium as gym
import numpy as np
import torch
from collections import defaultdict

import torch.nn as nn
import torch.optim as optim

import random

In [ ]:
# Create the environment
env = gym.make("FrozenLake-v1", is_slippery=False)  # deterministic version

# Define parameters
num_episodes = 5000
gamma = 0.99  # discount factor
learning_rate = 0.1
alpha = 0.1  # learning rate

state_size = env.observation_space.n

In [ ]:
# Monte Carlo

In [ ]:
# Initialize value function as torch tensor
value_table = torch.zeros(state_size, dtype=torch.float32)

# Helper: generate an episode
def generate_episode(env):
    episode = []
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()  # random policy
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode.append((state, action, reward))
        state = next_state
    return episode

# Monte Carlo Prediction
returns_sum = defaultdict(float)
returns_count = defaultdict(int)

for episode_idx in range(num_episodes):
    episode = generate_episode(env)
    visited_states = set()
    G = 0
    for t in reversed(range(len(episode))):
        state, action, reward = episode[t]
        G = gamma * G + reward
        if state not in visited_states:
            visited_states.add(state)
            returns_sum[state] += G
            returns_count[state] += 1
            value_table[state] = returns_sum[state] / returns_count[state]

# Print the learned value function
print("Learned Value Function (state-values):")
for s in range(state_size):
    print(f"State {s}: {value_table[s].item():.3f}")


Learned Value Function (state-values):
State 0: 0.017
State 1: 0.016
State 2: 0.032
State 3: 0.015
State 4: 0.018
State 5: 0.000
State 6: 0.066
State 7: 0.000
State 8: 0.039
State 9: 0.107
State 10: 0.198
State 11: 0.000
State 12: 0.000
State 13: 0.258
State 14: 0.514
State 15: 0.000


In [ ]:
# TD learning

In [ ]:
# Initialize value function as torch tensor
value_table = torch.zeros(state_size, dtype=torch.float32)
# Value function (as a torch tensor)
state_size = env.observation_space.n
value_table = torch.zeros(state_size, dtype=torch.float32)

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False

    while not done:
        action = env.action_space.sample()  # random policy
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # TD(0) update rule: V(s) ← V(s) + α [R + γ * V(s') - V(s)]
        td_target = reward + gamma * value_table[next_state]
        td_error = td_target - value_table[state]
        value_table[state] += alpha * td_error

        state = next_state

# Print the learned value function
print("Learned Value Function (TD):")
for s in range(state_size):
    print(f"State {s}: {value_table[s].item():.3f}")

Learned Value Function (TD):
State 0: 0.007
State 1: 0.005
State 2: 0.008
State 3: 0.006
State 4: 0.010
State 5: 0.000
State 6: 0.011
State 7: 0.000
State 8: 0.025
State 9: 0.051
State 10: 0.094
State 11: 0.000
State 12: 0.000
State 13: 0.098
State 14: 0.205
State 15: 0.000


In [ ]:
# TD learning: neural network

In [ ]:
# One-hot encoding for discrete states
def one_hot(state, num_states):
    vec = torch.zeros(num_states, dtype=torch.float32)
    vec[state] = 1.0
    return vec

# Define the value network
class ValueNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # Output: V(s)
        )

    def forward(self, x):
        return self.net(x)

# Initialize network and optimizer
value_net = ValueNetwork(state_size)
optimizer = optim.Adam(value_net.parameters(), lr=0.01)

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False

    while not done:
        action = env.action_space.sample()  # random policy
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        # Prepare inputs
        s = one_hot(state, state_size)
        s_next = one_hot(next_state, state_size)

        # Compute TD target: R + γ * V(s')
        with torch.no_grad():
            target = reward + gamma * value_net(s_next)

        # Prediction and loss
        prediction = value_net(s)
        loss = nn.functional.mse_loss(prediction, target)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state

# Show learned value estimates
print("State-Value Estimates (using neural network):")
for s in range(state_size):
    v = value_net(one_hot(s, state_size)).item()
    print(f"State {s}: {v:.3f}")

State-Value Estimates (using neural network):
State 0: 0.008
State 1: 0.007
State 2: 0.013
State 3: 0.021
State 4: 0.008
State 5: -0.008
State 6: 0.009
State 7: 0.035
State 8: 0.019
State 9: -0.002
State 10: -0.014
State 11: -0.008
State 12: 0.037
State 13: -0.003
State 14: -0.040
State 15: 0.006


TD learning: sarsa algorithm

In [ ]:
import gymnasium as gym
import numpy as np

# Create environment
env = gym.make("FrozenLake-v1", is_slippery=False)  # deterministic for easier learning
n_states = env.observation_space.n
n_actions = env.action_space.n

# Initialize Q-table
Q = np.zeros((n_states, n_actions))
alpha = 0.1       # learning rate
gamma = 0.99      # discount factor
epsilon = 0.1     # exploration rate

# Epsilon-greedy policy
def epsilon_greedy(state, Q, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(n_actions)
    return np.argmax(Q[state])

# Training loop
num_episodes = 5000
for episode in range(num_episodes):
    state, _ = env.reset()
    action = epsilon_greedy(state, Q, epsilon)
    done = False

    while not done:
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_action = epsilon_greedy(next_state, Q, epsilon)

        # SARSA update
        td_target = reward + gamma * Q[next_state, next_action]
        td_error = td_target - Q[state, action]
        Q[state, action] += alpha * td_error

        state = next_state
        action = next_action

# Print the learned Q-values
print("Learned Q-values (SARSA):")
print(Q)

# Derive policy
policy = np.argmax(Q, axis=1)
print("Derived policy:")
print(policy.reshape((4, 4)))  # 4x4 grid for FrozenLake

Learned Q-values (SARSA):
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Derived policy:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


TD learning: sarsa, neural network version
https://www.bilibili.com/video/BV12o4y197US?spm_id_from=333.788.videopod.episodes&vd_source=1fecee762931e992c96e5e166be13b76&p=8

In [ ]:
# Set up environment
env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Q-network
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )

    def forward(self, x):
        return self.net(x)

# Initialize network and optimizer
q_net = QNetwork(state_dim, action_dim)
optimizer = optim.Adam(q_net.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Hyperparameters
epsilon = 0.1
gamma = 0.99
num_episodes = 500
batch_size = 1  # SARSA is typically updated per step

def select_action(state, epsilon):
    if random.random() < epsilon:
        return random.randint(0, action_dim - 1)
    else:
        with torch.no_grad():
            q_values = q_net(torch.FloatTensor(state))
            return torch.argmax(q_values).item()

# Training loop
for episode in range(num_episodes):
    state, _ = env.reset()
    action = select_action(state, epsilon)
    done = False
    total_reward = 0

    while not done:
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_action = select_action(next_state, epsilon)

        # Compute target using SARSA formula
        q_values_next = q_net(torch.FloatTensor(next_state))
        target = reward + (gamma * q_values_next[next_action].item() * (not done))

        # Predicted Q(s, a)
        q_values = q_net(torch.FloatTensor(state))
        predicted = q_values[action]

        loss = loss_fn(predicted, torch.tensor(target))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state
        action = next_action
        total_reward += reward

    if episode % 50 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")

env.close()

Episode 0, Total Reward: 9.0
Episode 50, Total Reward: 10.0
Episode 100, Total Reward: 9.0
Episode 150, Total Reward: 15.0
Episode 200, Total Reward: 12.0
Episode 250, Total Reward: 14.0
Episode 300, Total Reward: 18.0
Episode 350, Total Reward: 22.0
Episode 400, Total Reward: 20.0
Episode 450, Total Reward: 12.0


DQN

https://www.bilibili.com/video/BV12o4y197US?spm_id_from=333.788.videopod.episodes&vd_source=1fecee762931e992c96e5e166be13b76&p=9


In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# Create environment
env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Q-network and target network
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )

    def forward(self, x):
        return self.net(x)

# Initialize networks and optimizer
q_net = QNetwork(state_dim, action_dim)
target_net = QNetwork(state_dim, action_dim)
target_net.load_state_dict(q_net.state_dict())  # copy initial weights
target_net.eval()

optimizer = optim.Adam(q_net.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Hyperparameters
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.05
batch_size = 64
replay_buffer = deque(maxlen=10000)
target_update_freq = 10  # episodes

# Epsilon-greedy policy
def select_action(state, epsilon):
    if random.random() < epsilon:
        return random.randint(0, action_dim - 1)
    with torch.no_grad():
        return torch.argmax(q_net(torch.FloatTensor(state))).item()

# Training loop
num_episodes = 500
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = select_action(state, epsilon)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        replay_buffer.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        # Training step
        if len(replay_buffer) >= batch_size:
            batch = random.sample(replay_buffer, batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)

            states = torch.FloatTensor(states)
            actions = torch.LongTensor(actions)
            rewards = torch.FloatTensor(rewards)
            next_states = torch.FloatTensor(next_states)
            dones = torch.BoolTensor(dones)

            q_values = q_net(states).gather(1, actions.unsqueeze(1)).squeeze()
            with torch.no_grad():
                next_q_values = target_net(next_states).max(1)[0]
                targets = rewards + gamma * next_q_values * (~dones)

            loss = loss_fn(q_values, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Decay epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Update target network
    if episode % target_update_freq == 0:
        target_net.load_state_dict(q_net.state_dict())

    if episode % 20 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward:.1f}, Epsilon: {epsilon:.2f}")

env.close()

Episode 0, Total Reward: 12.0, Epsilon: 0.99


<ipython-input-1-3a580307a4e6>:72: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  states = torch.FloatTensor(states)


Episode 20, Total Reward: 12.0, Epsilon: 0.90
Episode 40, Total Reward: 20.0, Epsilon: 0.81
Episode 60, Total Reward: 21.0, Epsilon: 0.74
Episode 80, Total Reward: 14.0, Epsilon: 0.67
Episode 100, Total Reward: 45.0, Epsilon: 0.60
Episode 120, Total Reward: 16.0, Epsilon: 0.55
Episode 140, Total Reward: 158.0, Epsilon: 0.49
Episode 160, Total Reward: 227.0, Epsilon: 0.45
Episode 180, Total Reward: 71.0, Epsilon: 0.40
Episode 200, Total Reward: 218.0, Epsilon: 0.37
Episode 220, Total Reward: 302.0, Epsilon: 0.33
Episode 240, Total Reward: 266.0, Epsilon: 0.30
Episode 260, Total Reward: 181.0, Epsilon: 0.27
Episode 280, Total Reward: 203.0, Epsilon: 0.24
Episode 300, Total Reward: 173.0, Epsilon: 0.22
Episode 320, Total Reward: 120.0, Epsilon: 0.20
Episode 340, Total Reward: 129.0, Epsilon: 0.18
Episode 360, Total Reward: 128.0, Epsilon: 0.16
Episode 380, Total Reward: 128.0, Epsilon: 0.15
Episode 400, Total Reward: 132.0, Epsilon: 0.13
Episode 420, Total Reward: 137.0, Epsilon: 0.12
Epi